In [4]:
import torch
import math
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
words = open('names.txt').read().splitlines()
print(len(words))

32033


In [3]:
t = {}
for w in words[:10]:
    chs = ['.', '.'] + list(w) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        t[(ch1,ch2,ch3)] = t.get((ch1,ch2,ch3), 0) + 1
print(t)

{('.', '.', 'e'): 2, ('.', 'e', 'm'): 1, ('e', 'm', 'm'): 1, ('m', 'm', 'a'): 1, ('m', 'a', '.'): 1, ('.', '.', 'o'): 1, ('.', 'o', 'l'): 1, ('o', 'l', 'i'): 1, ('l', 'i', 'v'): 1, ('i', 'v', 'i'): 1, ('v', 'i', 'a'): 1, ('i', 'a', '.'): 4, ('.', '.', 'a'): 2, ('.', 'a', 'v'): 1, ('a', 'v', 'a'): 1, ('v', 'a', '.'): 1, ('.', '.', 'i'): 1, ('.', 'i', 's'): 1, ('i', 's', 'a'): 1, ('s', 'a', 'b'): 1, ('a', 'b', 'e'): 1, ('b', 'e', 'l'): 1, ('e', 'l', 'l'): 1, ('l', 'l', 'a'): 1, ('l', 'a', '.'): 1, ('.', '.', 's'): 1, ('.', 's', 'o'): 1, ('s', 'o', 'p'): 1, ('o', 'p', 'h'): 1, ('p', 'h', 'i'): 1, ('h', 'i', 'a'): 1, ('.', '.', 'c'): 1, ('.', 'c', 'h'): 1, ('c', 'h', 'a'): 1, ('h', 'a', 'r'): 2, ('a', 'r', 'l'): 1, ('r', 'l', 'o'): 1, ('l', 'o', 't'): 1, ('o', 't', 't'): 1, ('t', 't', 'e'): 1, ('t', 'e', '.'): 1, ('.', '.', 'm'): 1, ('.', 'm', 'i'): 1, ('m', 'i', 'a'): 1, ('.', 'a', 'm'): 1, ('a', 'm', 'e'): 1, ('m', 'e', 'l'): 1, ('e', 'l', 'i'): 1, ('l', 'i', 'a'): 1, ('.', '.', 'h'): 1,

In [36]:
sorted(t.items(), key = lambda kv : -kv[1])

[(('i', 'a', '.'), 4),
 (('.', '.', 'e'), 2),
 (('.', '.', 'a'), 2),
 (('h', 'a', 'r'), 2),
 (('.', 'e', 'm'), 1),
 (('e', 'm', 'm'), 1),
 (('m', 'm', 'a'), 1),
 (('m', 'a', '.'), 1),
 (('.', '.', 'o'), 1),
 (('.', 'o', 'l'), 1),
 (('o', 'l', 'i'), 1),
 (('l', 'i', 'v'), 1),
 (('i', 'v', 'i'), 1),
 (('v', 'i', 'a'), 1),
 (('.', 'a', 'v'), 1),
 (('a', 'v', 'a'), 1),
 (('v', 'a', '.'), 1),
 (('.', '.', 'i'), 1),
 (('.', 'i', 's'), 1),
 (('i', 's', 'a'), 1),
 (('s', 'a', 'b'), 1),
 (('a', 'b', 'e'), 1),
 (('b', 'e', 'l'), 1),
 (('e', 'l', 'l'), 1),
 (('l', 'l', 'a'), 1),
 (('l', 'a', '.'), 1),
 (('.', '.', 's'), 1),
 (('.', 's', 'o'), 1),
 (('s', 'o', 'p'), 1),
 (('o', 'p', 'h'), 1),
 (('p', 'h', 'i'), 1),
 (('h', 'i', 'a'), 1),
 (('.', '.', 'c'), 1),
 (('.', 'c', 'h'), 1),
 (('c', 'h', 'a'), 1),
 (('a', 'r', 'l'), 1),
 (('r', 'l', 'o'), 1),
 (('l', 'o', 't'), 1),
 (('o', 't', 't'), 1),
 (('t', 't', 'e'), 1),
 (('t', 'e', '.'), 1),
 (('.', '.', 'm'), 1),
 (('.', 'm', 'i'), 1),
 (('m', 'i'

In [6]:
#frequency lookup : N - How many times a character occurs given the previous two characters 
chars = ['.'] + sorted(list(set(''.join(words))))
N = torch.ones([27, 27, 27], dtype=torch.int32) # smoothing
stoi = {c:i for i, c in enumerate(chars)}
itos = {v:k for k, v in stoi.items()}
for w in words:
    chs = ['.', '.'] + list(w) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        idx1 = stoi[ch1]
        idx2 = stoi[ch2]
        idx3 = stoi[ch3]
        N[idx1,idx2, idx3] += 1 

In [5]:
# manual seed for reproducibility
g = torch.Generator().manual_seed(2147483647)

In [6]:
for i in range(5):
    idx1, idx2 = 0, 0
    out=''
    while True:
        # extract count
        freq = N[idx1, idx2].float()

        # get probabailities
        probs = freq / freq.sum()
        
        # shift the context forward
        idx1 = idx2 

        # sample
        idx2 = torch.multinomial(probs, num_samples=1, replacement=True, generator=g).item()

        out += itos[idx2]

        if idx2==0:
            break

    print(out)


ce.
bra.
jalius.
rochityharlonimittain.
luwak.


In [7]:
# Calculate probs once instead of calculating everytime
P = N.float()
P /= P.sum(2, keepdims=True) # For each context (i, j), choose the next character i.e. 
# For fixed (i, j), sum over k equals 1. axis 0,1,2 is N[i,j,k]
# i,j are prev chars and k is the next char given i,j

In [8]:
g = torch.Generator().manual_seed(2147483647) # must be in the same cell as it should be ran every time to generate same o/p

for i in range(5):
    idx1, idx2 = 0, 0
    out=''
    while True:
        probs = P[idx1, idx2]
        
        # shift the context forward
        idx1 = idx2 

        # sample
        idx2 = torch.multinomial(probs, num_samples=1, replacement=True, generator=g).item()

        out += itos[idx2]

        if idx2==0:
            break

    print(out)


ce.
bra.
jalius.
rochityharlonimittain.
luwak.


In [9]:
# loss calculation
log_likelihood = 0
n=0
for w in words:
    chs = ['.','.'] + list(w) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:],chs[2:]):
        idx1 = stoi[ch1]
        idx2 = stoi[ch2]
        idx3 = stoi[ch3]
        
        prob = P[idx1, idx2, idx3]
        log_prob = torch.log(prob)
        log_likelihood += log_prob
        n+=1

nll = - log_likelihood
print(f'loglikelihood: {log_likelihood}')
print(f'negative loglikelihood: {nll}')
print((f'avg negative loglikelihood: {nll/n}'))


loglikelihood: -504653.0
negative loglikelihood: 504653.0
avg negative loglikelihood: 2.2119739055633545


In [10]:
# create a dataset
xs, ys = [], []
for w in words:
    chs = ['.', '.'] + list(w) + ['.']
    for ch1, ch2, ch3 in zip(chs,chs[1:],chs[2:]):
        idx1 = stoi[ch1]
        idx2 = stoi[ch2]
        idx3 = stoi[ch3]
        xs.append([idx1, idx2])
        ys.append(idx3)
xs = torch.tensor(xs)
ys = torch.tensor(ys)
print(xs.shape, ys.shape)

torch.Size([228146, 2]) torch.Size([228146])


In [11]:
import torch.nn.functional as F

In [12]:
xenc = F.one_hot(xs, num_classes=27).float()
print(xenc.shape)


torch.Size([228146, 2, 27])


In [13]:
xenc[0]

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [14]:
# concat or flatten to have 1 vector per sample for NN
xenc = xenc.view(len(xs), 54)
print(xenc.shape)

torch.Size([228146, 54])


In [15]:
xenc[0]

tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [16]:
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((54,27), requires_grad=True, generator=g)
logits = xenc @ W
logits

tensor([[ 2.4812,  1.2482,  0.3723,  ...,  0.3125,  2.3701,  3.1549],
        [ 0.5352,  0.2418, -0.2616,  ..., -1.6675,  0.6432,  1.0764],
        [ 0.8098,  1.5744,  0.0988,  ..., -0.9052, -0.2371,  3.2653],
        ...,
        [ 0.3543,  0.8678,  0.0411,  ..., -1.8267,  1.9362, -0.1628],
        [ 0.4902, -1.4578,  0.0746,  ..., -0.0480,  0.8478, -1.1147],
        [ 1.5952, -0.6920, -0.7784,  ..., -1.5724,  0.8434, -0.9350]],
       grad_fn=<MmBackward0>)

In [17]:
counts  = logits.exp()
probs = counts / counts.sum(1, keepdims=True)
probs

tensor([[0.0820, 0.0239, 0.0099,  ..., 0.0094, 0.0734, 0.1608],
        [0.0237, 0.0177, 0.0107,  ..., 0.0026, 0.0264, 0.0407],
        [0.0224, 0.0482, 0.0110,  ..., 0.0040, 0.0079, 0.2616],
        ...,
        [0.0230, 0.0385, 0.0168,  ..., 0.0026, 0.1119, 0.0137],
        [0.0108, 0.0015, 0.0071,  ..., 0.0063, 0.0154, 0.0022],
        [0.1024, 0.0104, 0.0095,  ..., 0.0043, 0.0483, 0.0082]],
       grad_fn=<DivBackward0>)

In [18]:
-probs.log().mean()

tensor(4.2447, grad_fn=<NegBackward0>)

In [ ]:
# complete version
xs, ys = [], []

for w in words:
    chs = ['.','.'] + list(w) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:],chs[2:]):
        idx1 = stoi[ch1]
        idx2 = stoi[ch2]
        idx3 = stoi[ch3]
        xs.append([idx1,idx2])
        ys.append(idx3)

xs = torch.tensor(xs)
ys = torch.tensor(ys)

print(xs.shape, ys.shape)

xenc = F.one_hot(xs, num_classes=27).float()
print(xenc.shape)

xenc = xenc.view(len(xs),54)
print('after concat:', xenc.shape)


#initialization
g = torch.Generator().manual_seed(456)
W = torch.randn((54,27), requires_grad=True, generator=g)


torch.Size([228146, 2]) torch.Size([228146])
torch.Size([228146, 2, 27])
after concat: torch.Size([228146, 54])


In [29]:
# gradient descent
for i in range(50):
    # forward pass
    logits = xenc @ W
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdims=True)
    loss = -probs[torch.arange(len(xs)),ys].log().mean()

    print(f'loss: {loss}')

    # backward pass
    W.grad = None
    loss.backward()

    #update
    W.data += -50*W.grad



loss: 2.44396710395813
loss: 2.3847761154174805
loss: 2.408766508102417
loss: 2.3935599327087402
loss: 2.443021059036255
loss: 2.383620500564575
loss: 2.4075112342834473
loss: 2.3925271034240723
loss: 2.4421226978302
loss: 2.3825576305389404
loss: 2.4063711166381836
loss: 2.3915653228759766
loss: 2.4412691593170166
loss: 2.3815758228302
loss: 2.4053313732147217
loss: 2.3906681537628174
loss: 2.440459966659546
loss: 2.380666494369507
loss: 2.4043774604797363
loss: 2.389829158782959
loss: 2.4396934509277344
loss: 2.37982177734375
loss: 2.403498649597168
loss: 2.3890433311462402
loss: 2.438967227935791
loss: 2.3790345191955566
loss: 2.402686595916748
loss: 2.3883070945739746
loss: 2.438279628753662
loss: 2.3782992362976074
loss: 2.4019320011138916
loss: 2.3876144886016846
loss: 2.4376280307769775
loss: 2.377610683441162
loss: 2.4012296199798584
loss: 2.386962890625
loss: 2.437009811401367
loss: 2.376964569091797
loss: 2.40057373046875
loss: 2.3863494396209717
loss: 2.4364242553710938
loss

In [30]:
# sampling
g = torch.Generator().manual_seed(2147483647)
for i in range(5):
    idx1, idx2 = 0,0
    out = ''
    while True:
        xenc = F.one_hot(torch.tensor([[idx1,idx2]]), num_classes=27).float()
        xenc = xenc.view(1, 54)
        logits = xenc @ W
        counts = logits.exp()
        probs = counts / counts.sum(1, keepdims=True)

        idx3 = torch.multinomial(probs, num_samples=1, replacement=True, generator=g).item()
        out+=itos[idx3]
        
        # shift context
        idx2 = idx3
        idx1  = idx2

        if idx3 == 0 :
            break
    print(out)

        

ce.
be.
memaluriale.
kayhamelilisttain.
lenak.


train, dev, test sets - 80%, 10%, 10%

In [4]:
import random

In [5]:
random.seed(42)
random.shuffle(words)  # remove ordering bias. fair eval and same distribution is the goal
n = len(words)
train_len = int(n * 0.8)
dev_len = int(n * 0.9)
train_set = words[:train_len]
dev_set = words[train_len:dev_len]
test_set = words[dev_len:]
print(f'{n} --> {len(train_set)} + {len(dev_set)} + {len(test_set)}')

32033 --> 25626 + 3203 + 3204


In [26]:
xs_tr, ys_tr = [], []
for w in train_set:
    chs = ['.','.'] + list(w) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:],chs[2:]):
        xs_tr.append([stoi[ch1],stoi[ch2]])
        ys_tr.append(stoi[ch3])
xs_tr = torch.tensor(xs_tr)
ys_tr = torch.tensor(ys_tr)

print(xs_tr.shape, ys_tr.shape)

xs_tr_enc = F.one_hot(xs_tr,num_classes=27).float()
xs_tr_enc = xs_tr_enc.view(len(xs_tr), 2*27)
print(xs_tr_enc.shape)

g = torch.Generator().manual_seed(2147483647)
W = torch.randn((54, 27),requires_grad=True, generator=g)


torch.Size([182625, 2]) torch.Size([182625])
torch.Size([182625, 54])


In [18]:
# gradient descent 

for k in range(500):

    # forward pass

    logits = xs_tr_enc @ W
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdims=True)
    loss = -probs[torch.arange(len(xs_tr_enc)), ys_tr].log().mean()
    # print(f'loss: {loss}')

    # backward pass

    W.grad = None
    loss.backward()

    # update

    W.data += -5 * W.grad

print(loss.item())



2.35335636138916


In [27]:
def build_trigram_xy(words):
    xs, ys = [], []
    for w in words:
        chs = ['.', '.'] + list(w) + ['.']
        for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
            xs.append([stoi[ch1], stoi[ch2]])
            ys.append(stoi[ch3])
    xs = torch.tensor(xs)
    ys = torch.tensor(ys)

    xenc = F.one_hot(xs, num_classes=27).float()
    xenc = xenc.view(len(xs), 54)

    return xenc, ys

@torch.no_grad()
def evaluate(xenc, ys, W):
    logits = xenc @ W
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdims=True)
    loss = - probs[(torch.arange(len(ys))),ys].log().mean()
    return loss

In [19]:
xenc_d, ys_d = build_trigram_xy(dev_set)
dev_loss = evaluate(xenc_d, ys_d, W)
print(dev_loss)

tensor(2.3530)


In [20]:
xenc_t, ys_t = build_trigram_xy(test_set)
test_loss = evaluate(xenc_t, ys_t, W)
print(test_loss)

tensor(2.3553)


Smoothing/regularization

In [21]:
def loss_reg(xenc, ys, W, lambda_):
    logits = xenc @ W
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdims=True)
    loss = - probs[(torch.arange(len(ys))),ys].log().mean()
    reg = (lambda_ * (W**2)).mean()
    return loss, loss+reg

In [ ]:
results={}
lambdas = [0.0, 1e-4, 1e-3, 1e-2, 1e-1, 1.0]
for lambda_ in lambdas:
    g = torch.Generator().manual_seed(2147483647)
    W = torch.randn((54, 27), requires_grad=True, generator = g)

    for k in range(1000):
        nll, loss = loss_reg(xs_tr_enc, ys_tr, W, lambda_)

        W.grad = None
        loss.backward()

        W.data += -20* W.grad

    dev_loss = evaluate(xenc_d, ys_d, W)
    results[lambda_] = dev_loss.item()

best_lambda = min(results, key=results.get)
print("Best lambda:", best_lambda)
print(results)


Best lambda: 0.0
{0.0: 2.341703414916992, 0.0001: 2.3417139053344727, 0.001: 2.3418095111846924, 0.01: 2.3431200981140137, 0.1: 2.3640191555023193, 1.0: 2.4882419109344482, 1.5: 2.532921552658081}


In [35]:
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((54, 27), requires_grad=True, generator = g)

# train again with best lambda value
for k in range(1000):
    nll, loss = loss_reg(xs_tr_enc, ys_tr, W, best_lambda)

    W.grad = None
    loss.backward()

    W.data += -20* W.grad

# evaluate on test
test_loss = evaluate(xenc_t, ys_t, W)
print('test loss: ', test_loss)

test loss:  tensor(2.3440)


one hot encoding alternative

In [15]:
# complete version
xs, ys = [], []

for w in words:
    chs = ['.','.'] + list(w) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:],chs[2:]):
        idx1 = stoi[ch1]
        idx2 = stoi[ch2]
        idx3 = stoi[ch3]
        xs.append([idx1,idx2])
        ys.append(idx3)

xs = torch.tensor(xs)
ys = torch.tensor(ys)

print(xs.shape, ys.shape)

# xenc = F.one_hot(xs, num_classes=27).float()
# xenc = xenc.view(len(xs),54)

#initialization
g = torch.Generator().manual_seed(456)
W = torch.randn((54,27), requires_grad=True, generator=g)



torch.Size([228146, 2]) torch.Size([228146])


In [10]:
xs

tensor([[ 0,  0],
        [ 0,  5],
        [ 5, 13],
        ...,
        [26, 25],
        [25, 26],
        [26, 24]])

In [17]:
xs.shape

torch.Size([228146, 2])

In [16]:
# gradient descent 

for k in range(500):

    # forward pass

    # logits = xs_tr_enc @ W
    logits = W[xs].sum(dim=1) # alternatively using sepearte Ws it can be done as W1[xs[:,0]] + W2[xs[:,1]]

    counts = logits.exp()
    probs = counts / counts.sum(1, keepdims=True)
    loss = -probs[torch.arange(len(ys)), ys].log().mean()
    # print(f'loss: {loss}')

    # backward pass

    W.grad = None
    loss.backward()

    # update

    W.data += -5 * W.grad

print(loss.item())



2.5215535163879395


cross entropy

In [ ]:
# gradient descent 

for k in range(500):

    # forward pass

    # logits = xs_tr_enc @ W
    logits = W[xs].sum(dim=1) # alternatively using sepearte Ws it can be done as W1[xs[:,0]] + W2[xs[:,1]]

    # counts = logits.exp()
    # probs = counts / counts.sum(1, keepdims=True)
    # loss = -probs[torch.arange(len(ys)), ys].log().mean()

    # last 3 lines can be implemented as cross entropy = softmax + nll
    loss = F.cross_entropy(logits, ys)

    # print(f'loss: {loss}')

    # backward pass

    W.grad = None
    loss.backward()

    # update

    W.data += -5 * W.grad

print(loss.item())



2.5060222148895264
